# Sesion 3: Taller de tecnicas de modelado computacional de fluidos

## Elementos Finitos: Metodo Galerkin Discontinuo

**Por: Erick Urquilla, Universidad de Tennessee, Knoxville, USA**

Resolveremos las ecuaciones que gobiernan la dinámica de fluidos poco profundos incompresibles. El fluido está limitado inferiormente y a los costados por superficies rígidas y superiormente sin restricción. Estas ecuaciones se derivan de las ecuaciones de Navier-Stokes cuando el dominio espacial horizontal del fluido excede significativamente al vertical, manteniendo el equilibrio hidrostático y densidad constante:

$$
\frac{\partial h}{\partial t} + \frac{\partial}{\partial x}\left(hu\right) = 0
$$
$$
\frac{\partial}{\partial t}\left(hu\right) + \frac{\partial}{\partial x}\left(hu^2 + \frac{gh^2}{2}\right) = 0
$$

Aquí, $h$ es la altura del fluido, $u$ es la velocidad horizontal y $g$ es la aceleración de la gravedad. El movimiento del fluido está dictado exclusivamente por las ecuaciones de conservación de masa y momento.

## Paquetes utilizados

En este proyecto, utilizaremos varios paquetes de Python que nos ayudarán a realizar diferentes tareas de manera eficiente. A continuación, se describen los paquetes que se utilizarán:

1. **numPy**
2. **matplotlib**
3. **IPython**
4. **time**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import time

## Scripts utilizados

En esta sesion importaremos los siguientes scripts

1. **bases.py**
2. **galerkin_discontinuo.py**
3. **paso_de_tiempo.py**
4. **graficos.py**

Puedes encontrar estos scripts en la misma ruta que este notebook

In [ ]:
# !rm -r 1D-Shallow-Water-Equation
# !git clone --branch curso_unah_sesion_3 --single-branch https://github.com/erickurquilla1999/1D-Shallow-Water-Equation.git
# !mv 1D-Shallow-Water-Equation/*.py ./

import sys
from pathlib import Path

aqui_camino = Path.cwd()
cdg_fuente_camino = (aqui_camino / '..' / 'cdg_fuente').resolve()
sys.path.append(str(cdg_fuente_camino))

import bases
import galerkin_discontinuo 
import paso_de_tiempo
import graficos

## Parámetros de entrada

1. Dominio espacial de la simulación
2. Número de elementos y número de nodos interiores por elemento
3. Tiempo de la simulación y número de pasos
4. Número de puntos en la cuadratura de Gauss para integración numérica

In [ ]:
# Parametros de entrada

# Dominio espacial de la simulacion 
x_inicial = 0 # (m) cooordenda inicial del dominio
x_final = 10 # (m) coordenada final del dominio

# Parametros del metodo de elementos finitos
N_elementos = 6 # numero de elementos
N_nodos = 4 # numero de nodos por elemento (por simplicidad solo consideramos N_nodos >= 2)

# Dominio temporal 
n_pasos = 100/20 # numero de pasos temporales
t_total = 1/20 # (s) tiempo final

# Integracion numerica con cuadratura de Gauss
n_nodos_cuadratura_gauss = 20

In [ ]:
# Generando malla
longitud_elemento = (x_final - x_inicial) / N_elementos
malla = np.array([np.linspace(x_inicial + i * (x_final - x_inicial) / N_elementos, x_inicial + (i + 1) * (x_final - x_inicial) / N_elementos, N_nodos) for i in range(N_elementos)])      

# Condiciones iniciales
h = 1.0 + 0.1 * np.exp( - ( malla - 5.0 )**2 ) # Altura (m)
u = malla * 0.0 # Velocidad horizontal (m/s)

In [ ]:
cuadratura_de_gauss, pesos_de_gauss = np.polynomial.legendre.leggauss(n_nodos_cuadratura_gauss)
polinomios_de_lagrange_en_cuadratura_de_gauss, derivada_x_polinomios_de_lagrange_en_cuadratura_de_gauss = bases.generate_reference_space(malla, cuadratura_de_gauss)
matriz_de_masa_inversa = galerkin_discontinuo.calcula_inversa_matriz_de_masa(longitud_elemento, pesos_de_gauss, polinomios_de_lagrange_en_cuadratura_de_gauss, N_nodos)
matriz_de_rigidez = galerkin_discontinuo.calcula_matrix_de_rigidez(longitud_elemento, pesos_de_gauss, polinomios_de_lagrange_en_cuadratura_de_gauss, derivada_x_polinomios_de_lagrange_en_cuadratura_de_gauss)

In [ ]:
# time step
time_step = np.array(t_total/n_pasos) 

# evolving in time the PDE
for number_of_t_step in np.arange(n_pasos):

    # calculando el vector residual
    vector_residual_1, vector_residual_2 = paso_de_tiempo.calcular_vector_residual(h, u, matriz_de_rigidez)

    # -----------------------------------------------------------------------------------------
    # Escribe tu solucion al ejercicio 5 a continuacion ...

    # inicializando los valores de d1U/dt y d2U/dt
    d1U_dt = np.zeros((N_elementos, N_nodos))
    d2U_dt = np.zeros((N_elementos, N_nodos))

    # calculando la derivada temporal de 1U y 2U ...
    # ... Escribe aqui d1U_dt y d2U_dt ... 
    # ... Haz un loop sobre cada elemento ...
    # ... Utiliza la matriz de masa inversa y el vector residual en cada elemento para calcular las derivadas ...
    for i in range(N_elementos):
        d1U_dt[i] = matriz_de_masa_inversa @ vector_residual_1[i]
        d2U_dt[i] = matriz_de_masa_inversa @ vector_residual_2[i]

    # inicializando los valores de dh/dt y du/dt
    dh_dt = np.zeros((N_elementos, N_nodos))
    du_dt = np.zeros((N_elementos, N_nodos))

    # calculando la derivada temporal de h y u
    # ... Escribe aqui dh_dt y du_dt ...
    # ... recuerda que 1U=h y 2U=hu por lo que du_dt = ( d2U_dt - u * dh_dt ) / h ...
    dh_dt = d1U_dt
    du_dt = np.where( h == 0 , 0 , ( d2U_dt - u * dh_dt ) / h )

    # inicializando los valores de h y u en el siguiente paso de tiempo
    h_nuevo = np.zeros((N_elementos, N_nodos))
    u_nuevo = np.zeros((N_elementos, N_nodos))

    # calculando los valores de h y u en el siguiente paso de tiempo con el metodo de Euler explicito
    # ... Escribe aqui h_nuevo y u_nuevo dados por el metodo de Euler explicito ...
    h_nuevo = h + dh_dt * time_step
    u_nuevo = u + du_dt * time_step
    
    # -----------------------------------------------------------------------------------------

    # graficando la simulacion
    graficos.plot_simulation(malla, h_nuevo, u_nuevo, N_elementos, time_step, number_of_t_step, display=True)

    # actualizando los valores de h y u
    h = h_nuevo
    u = u_nuevo

print(f'Done')